In [1]:


import random
values={'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, 
         '9': 9, '10': 10, 'J': 10, 'Q': 10, 'K': 10, 'A': 11}

def new_deck(nr):
    deck=['A', '2','3','4','5','6','7','8','9','10','J','Q','K']*4*nr
    random.shuffle(deck)
    return deck
    
def add_card(player, deck):
    new_card=deck.pop()
    player.append(new_card)
    
def card_value(card):
    return values[card]

def print_card(card):
    if card!='10':
        card_image="""┌───────┐
│ {}     │
│       │
│       │
│     {} │
└───────┘""".format(card, card)

    else:
        card_image="""┌───────┐
│ {}    │
│       │
│       │
│    {} │
└───────┘""".format(card, card)
    print(card_image)

def dealer_turn(deck, dealer):
    summa=card_value(dealer[0])+card_value(dealer[1])
    if summa==21:
        return "blackjack"
    while summa<=16:
        add_card(dealer, deck)
        summa+=card_value(dealer[-1])
    if summa>21:
        if 'A' in dealer:
            summa-=10
    return summa

In [2]:
def split(player_deck, deck, money, bet):      #resplit változó, szerintem csinálhatjuk a legegyszrűbbet először, hogy nem lehet
    
    if card_value(player_deck[0]) == card_value(player_deck[1]) and money[0]>=bet*2:
        choice = input("\n Do [y]ou wa[n]t to split? ").lower()
            
        if choice == "y":
            
            player_deck_new1 = []
            player_deck_new2 = []
            
            player_deck_new1.append(player_deck[0])
            add_card(player_deck_new1, deck)
                                 
            player_deck_new2.append(player_deck[1])
            add_card(player_deck_new2, deck)  
            
            return [1, player_deck_new1, player_deck_new2]
                                    
        elif choice == "n":
            return [0, player_deck]
        
    else:
        return [0, player_deck]

In [3]:
def player_turn(deck, player, money, bet):
    summa=card_value(player[0])+card_value(player[1])
    if summa==21:
        return ["blackjack", 0]        # 0: nem volt duplázás, 1: volt duplázás
    
    #itt van a double down (kiszervezzem-e külön függvénybe?)
    
    if 2*bet <= money[0]:
    
        double_dec = input("Do [y]ou wa[n]t to double down?").lower()  
        if double_dec == "y":
            add_card(player, deck)                 #meg lehetne csinálni egy kérdéssel: [D]ouble, [H]int,[S]tand
            print_card(player[-1])
            summa+=card_value(player[-1])
            if summa>21:
                    for i in range(len(player)):
                        if player[i]=='A':
                            player[i]="A_as1"
                            summa-=10
                            break
                    if summa>21:
                        print("Bust")
            return [summa, 1]
        
        elif double_dec == "n":
            while 1:
                if summa==21:
                    return [21, 0]
                decision=input("Choose: [h]it of [s]tand? ").lower()
                if decision=="s":
                    return [summa, 0]
                elif decision=="h":
                    add_card(player, deck)
                    print_card(player[-1])
                    summa+=card_value(player[-1])
                    if summa>21:
                        for i in range(len(player)):
                            if player[i]=='A':
                                player[i]="A_as1"
                                summa-=10
                                break
                        if summa>21:
                            print("Bust")
                            return [summa, 0]
                else:
                    continue
        
            

    else:        
        while 1:
            if summa==21:
                return [21, 0]
            decision=input("Choose: [h]it of [s]tand? ").lower()
            if decision=="s":
                return [summa, 0]
            elif decision=="h":
                add_card(player, deck)
                print_card(player[-1])
                summa+=card_value(player[-1])
                if summa>21:
                    for i in range(len(player)):
                        if player[i]=='A':
                            player[i]="A_as1"
                            summa-=10
                            break
                    if summa>21:
                        print("Bust")
                        return [summa, 0]
            else:
                continue
        


In [4]:
def game(deck, nr, money1, bet1, win, nr_game):
    dealer=[]
    player=[]
    add_card(player, deck)
    add_card(dealer, deck)
    add_card(player, deck)
    add_card(dealer, deck)        
    print("Your cards:")
    print_card(player[0])
    print_card(player[1])
    print("Dealer's first card: ")
    print_card(dealer[0])
    
    player_new = split(player, deck, money1, bet1)
    
    if player_new[0] == 1:
        remaining_game=[player_new[1], player_new[2]]
        
    elif player_new[0] == 0:    
        remaining_game=[player_new[1]]
    
    returnlist=[]
    answers=[]
    is_doubled1 = 0
    money2 = []
    money2.append(money1[0])
    for i in remaining_game:            
        if player_new[0]==1:            
            print("Your cards in this split:") 
            print_card(i[0])
            print_card(i[1])
        end_dealer=dealer_turn(deck, dealer)
        if i == 1 and is_doubled1 == 1:   #megnézni hogy elsö körben double down volt-e
            end_list=game_2(deck, nr, [money2[0]-2*bet1], bet1, i, end_dealer, dealer)  
        else:
            end_list=game_2(deck, nr, [money2[0]-(len(remaining_game)-1)*bet1], bet1, i, end_dealer, dealer) 
        end = end_list[0]                                                
        is_double = end_list[1]
        is_doubled1 = end_list[1]
        nr_game[0]+=1
        if end=="T":
            answers.append("Tie :|")
        elif end=="W":
            answers.append("You won :)")
            if is_double == 0:
                money1[0]+=bet1
            elif is_double == 1:
                money1[0] = money1[0] + 2*bet1
            win[0]+=1
        else:
            answers.append("You lost :(")
            if is_double == 0:
                money1[0]-=bet1
            elif is_double == 1:
                money1[0] = money1[0] - 2*bet1
    print("Dealer's turn")
    for i in range(1, len(dealer)):
        print_card(dealer[i])
    if player_new[0] == 1:
        print("Results in your splits:")
    for i in answers:
        print(i)
                

In [5]:
def game_2(deck, nr, money, bet, player, end_dealer, dealer):
        end_player_list = player_turn(deck, player, money, bet)
        end_player = end_player_list[0]
        is_double = end_player_list[1]
        if end_player!="blackjack" and end_player>21:
            return ["L", is_double]
        #print("Dealer's turn")
        #for i in range(1, len(dealer)):
        #    print_card(dealer[i])
        if end_player=="blackjack":
            if end_dealer=="blackjack":
                return ["T", is_double]
            else:
                return ["W", is_double]
        elif end_dealer=="blackjack":
            return ["L", is_double]
        elif end_dealer>21:
            return ["W", is_double]
        elif end_player>end_dealer:
            return ["W", is_double]
        elif end_player<end_dealer:
            return ["L", is_double]
        else:
            return ["T", is_double]

In [6]:
def start_playing():
    nr=int(input("Number of decks? "))
    deck=new_deck(nr)
    money=[int(input("Starting amount?"))]
    minbet=int(input("Minimum bet?"))
    nr_game=[0]
    win=[0]
    while money[0]>0:
        if len(deck)<10:
            print("Just a few cards left, rescuffling")
            deck=new_deck(nr)
        if nr_game[0]!=0:
            new=input("\n Do [y]ou want to play o[n]e more time?")
            if new=="n":
                break
        while 1:
            bet=int(input("Your bet? "))
            if bet>=minbet and bet<=money[0]:
                break
            else:
                print("Invalid value")
        game(deck, nr,  money, bet, win, nr_game)
            
            
            
            
    print("\n You played ", nr_game[0], " games, won ", win[0], " out of these.")
    print("Your balance is ", money[0], ".")
    
start_playing()

Number of decks? 4
Starting amount?100
Minimum bet?10
Your bet? 10
Your cards:
┌───────┐
│ 2     │
│       │
│       │
│     2 │
└───────┘
┌───────┐
│ Q     │
│       │
│       │
│     Q │
└───────┘
Dealer's first card: 
┌───────┐
│ 5     │
│       │
│       │
│     5 │
└───────┘
Do [y]ou wa[n]t to double down?n
Choose: [h]it of [s]tand? h
┌───────┐
│ 2     │
│       │
│       │
│     2 │
└───────┘
Choose: [h]it of [s]tand? h
┌───────┐
│ 5     │
│       │
│       │
│     5 │
└───────┘
Choose: [h]it of [s]tand? s
Dealer's turn
┌───────┐
│ Q     │
│       │
│       │
│     Q │
└───────┘
┌───────┐
│ Q     │
│       │
│       │
│     Q │
└───────┘
You won :)

 Do [y]ou want to play o[n]e more time?n

 You played  1  games, won  1  out of these.
Your balance is  110 .
